In [0]:
sample_column_names = ["ItemName", "Value"]
sample_data = [["Item1", 1.0], ["Item2", 2.0], ["Item3", 3.0], ["Item4", 4.0]]

df = spark.createDataFrame(sample_data, sample_column_names)

### Write to ADLS
In Databricks it is easiest to mount the ADLS location. You can write to local filesystem or specially mounted folders depending on the environment you are in. Many different protocols that may require their own Spark extensions to be installed.

Common protocols:
- dbfs:/     Databricks filesystem
- file://    Local filesystem
- hdfs://    Hadoop filesystem
- abfss://   Azure Data Lake Storage (Gen2)
- wasbs://   Azure Blob Storage
- s3://      AWS S3 (several variations of this)
- gs://      Google Cloud Storage

In [0]:
# CSV
csv_path = "/mnt/adlsdemo/write_examples/items_csv"
(
  df.write
    .mode("overwrite")
    .option("header","true")
    .csv(csv_path) 
 )

In [0]:
# Parquet
parquet_path = "/mnt/adlsdemo/write_examples/items_parquet"

(
  df.write
    .mode("overwrite")
    .parquet(parquet_path)
)

In [0]:
# Delta
delta_path = "/mnt/adlsdemo/write_examples/items_delta"

(
  df.write
    .format("delta")
    .save(delta_path)
)

### Write to JDBC

In [0]:
database = "StackOverflow2010"
db_host_name = "sandbox-2-sqlserver.database.windows.net"
db_url = f"jdbc:sqlserver://{db_host_name};databaseName={database}"
db_user = dbutils.secrets.get("demo", "sql-user-stackoverflow") # databricks
db_password = dbutils.secrets.get("demo", "sql-pwd-stackoverflow") #databricks
  
table = "items_test"

(
  df.write
    .mode("overwrite")
    .format("jdbc")
    .option("url", db_url)
    .option("dbtable", table)
    .option("user", db_user)
    .option("password", db_password)
    .save()
)

### Write to SQL Server (special driver)

In [0]:
  # On Databricks, need to add library for com.microsoft.azure:spark-mssql-connector_2.12:1.2.0 and set secrets
database = "StackOverflow2010"
db_host_name = "sandbox-2-sqlserver.database.windows.net"
db_url = f"jdbc:sqlserver://{db_host_name};databaseName={database}"
db_user = dbutils.secrets.get("demo", "sql-user-stackoverflow") # databricks
db_password = dbutils.secrets.get("demo", "sql-pwd-stackoverflow") #databricks

table = "items_test2"

(
    df.write
    .mode("overwrite")
    .format("com.microsoft.sqlserver.jdbc.spark")
    .option("url", db_url)
    .option("dbtable", table)
    .option("user", db_user)
    .option("password", db_password)
    .save()
)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1972616882368315> in <module> 
 9 
 10 (
 ---> 11 df . write
 12 . mode ( "overwrite" ) 
 13 . format ( "com.microsoft.sqlserver.jdbc.spark" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 736 self . format ( format ) 
 737 if path is None : 
 --> 738 self . _jwrite . save ( ) 
 739 else : 
 740 self . _jwrite . save ( path ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o688.save.
: java.lang.ClassNotFoundException: 
Failed to find data source: com.microsoft.sqlserver.jdbc.spark. Please find packages at
http://spark.apache.org/third-party-projects.html
 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:557)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:758)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:808)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:983)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:293)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.spark.DefaultSource
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:419)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:352)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$lookupDataSource$5(DataSource.scala:744)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$lookupDataSource$4(DataSource.scala:744)
	at scala.util.Failure.orElse(Try.scala:224)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:744)
	... 15 more

### Write to Snowflake

In [0]:
user = dbutils.secrets.get("demo", "snowflake-user")
password = dbutils.secrets.get("demo", "snowflake-password")
snowflake_url = "https://ux22775.west-us-2.azure.snowflakecomputing.com/"
snowflake_database = "CITIBIKE"
snowflake_schema = "PUBLIC"
snowflake_cluster = "COMPUTE_WH"

# snowflake connection options
options = {
  "sfUrl": snowflake_url,
  "sfUser": user,
  "sfPassword": password,
  "sfDatabase": snowflake_database,
  "sfSchema": snowflake_schema,
  "sfWarehouse": snowflake_cluster
}

df.write \
  .format("snowflake") \
  .options(**options) \
  .option("dbtable", "ITEMS_TEST") \
  .save()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2006543728526086> in <module> 
 16 }
 17 
 ---> 18 df = spark . read \ 
 19 . format ( "snowflake" ) \ 
 20 . options ( ** options ) \ 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 162 return self . _df ( self . _jreader . load ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 163 else : 
 --> 164 return self . _df ( self . _jreader . load ( ) ) 
 165 
 166 def json(self, path, schema=None, primitivesAsString=None, prefersDecimal=None,

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o915.load.
: net.snowflake.client.jdbc.SnowflakeSQLException: Your account is suspended due to lack of payment method.
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionSub(SnowflakeUtil.java:126)
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowException(SnowflakeUtil.java:66)
	at net.snowflake.client.core.StmtUtil.pollForOutput(StmtUtil.java:434)
	at net.snowflake.client.core.StmtUtil.execute(StmtUtil.java:338)
	at net.snowflake.client.core.SFStatement.executeHelper(SFStatement.java:482)
	at net.snowflake.client.core.SFStatement.executeQueryInternal(SFStatement.java:194)
	at net.snowflake.client.core.SFStatement.executeQuery(SFStatement.java:131)
	at net.snowflake.client.core.SFStatement.execute(SFStatement.java:737)
	at net.snowflake.client.core.SFStatement.execute(SFStatement.java:635)
	at net.snowflake.client.jdbc.SnowflakeStatementV1.executeQueryInternal(SnowflakeStatementV1.java:238)
	at net.snowflake.client.jdbc.SnowflakePreparedStatementV1.executeQuery(SnowflakePreparedStatementV1.java:117)
	at net.snowflake.spark.snowflake.JDBCWrapper.$anonfun$executePreparedQueryInterruptibly$1(SnowflakeJDBCWrapper.scala:330)
	at net.snowflake.spark.snowflake.JDBCWrapper.$anonfun$executeInterruptibly$2(SnowflakeJDBCWrapper.scala:368)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)